In [16]:
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

import pandas as pd
import json
from xgboost import XGBRegressor
import geopy
from geopy import distance
from geopy.distance import vincenty
# from tqdm import tqdm
# tqdm.pandas()
import seaborn as sns
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV
from sklearn.svm import NuSVR, SVR
from sklearn.kernel_ridge import KernelRidge
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
from scipy import stats
from dateutil import parser
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
import numpy as np
from scipy import stats

import eli5
from eli5.sklearn import PermutationImportance
# from skopt.space import Real
import matplotlib.colors as clt
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
# from tqdm import tqdm
from scipy.stats import kurtosis, skew

from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR
from sklearn.metrics import mean_absolute_error
%matplotlib inline
from keras.layers import Dense
# Change pandas viewing options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zipfile
from keras.models import Sequential
# zf = zipfile.ZipFile('train.csv') # having First.csv zipped file.
# df = pd.read_csv(zf.open('First.csv'))
# zf.namelist()

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [17]:
def gen_features(X):
    strain = []
    x1=[]
    strain.append(X.mean())
    strain.append(X.std())
    strain.append(X.min())
    strain.append(X.max())
    strain.append(X.kurtosis())
    strain.append(X.skew())
    strain.append(X.ptp())
    strain.append(X.var())
    zc=np.fft.fft(X)
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    strain.append(realFFT.mean())
    strain.append(realFFT.std())
    strain.append(realFFT.max())
    strain.append(realFFT.min())
    strain.append(imagFFT.mean())
    strain.append(imagFFT.std())
    strain.append(imagFFT.max())
    strain.append(imagFFT.min())
#   X_train1.columns = ['mean','std', 'min', 'max', 'kurt', 'skew','ptp','var','Rmean','Rstd','Rmax','Rmin','Imean','Istd','Imax','Imin','sum','mad', 'median','q01','q95','q99','q05']

    strain.append(X.sum())
    strain.append(X.mad())
    strain.append(X.median())
    strain.append(np.quantile(X, 0.01))
    strain.append(np.quantile(X, 0.95))
    strain.append(np.quantile(X, 0.99))
    strain.append(np.quantile(X, 0.05))
    
#     stats1=stats.binned_statistic([1, 3, 5, 7], X, 'mean', bins=5)
#     strain.append(stats)
    return pd.Series(strain)

# final.describe()
  

In [18]:
train = pd.read_csv('train.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})


In [19]:

X_train1 = pd.DataFrame()
y_train1= pd.Series()
for df in train:
    ch = gen_features(df['acoustic_data'])
    X_train1 = X_train1.append(ch, ignore_index=True)
    y_train1 = y_train1.append(pd.Series(df['time_to_failure'].values[-1]))

In [20]:
X_train1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,4.884113,5.101106,-98.0,104.0,33.662481,-0.024061,202.0,26.021284,12.0,2349.811482,732617.0,-20121.154171,-1.697723e-15,1399.854635,23432.719433,-23432.719433,732617.0,3.263401,5.0,-8.0,11.0,18.0,-2.0
1,4.725767,6.588824,-154.0,181.0,98.758517,0.390561,335.0,43.412599,5.0,2566.032248,708865.0,-31056.675076,1.843243e-15,1810.312266,27236.180586,-27236.180586,708865.0,3.574302,5.0,-11.0,12.0,21.0,-2.0
2,4.906393,6.967397,-106.0,140.0,33.555211,0.217391,246.0,48.544621,5.0,2683.549049,735959.0,-27654.557067,4.850638e-16,1921.220576,30073.497066,-30073.497066,735959.0,3.948411,5.0,-15.0,13.0,26.0,-3.0
3,4.902240,6.922305,-199.0,197.0,116.548172,0.757278,396.0,47.918309,5.0,2685.788525,735336.0,-25622.393604,-1.067140e-15,1891.826366,27380.321471,-27380.321471,735336.0,3.647117,5.0,-12.0,12.0,22.0,-2.0
4,4.908720,7.301110,-126.0,145.0,52.977905,0.064531,271.0,53.306210,12.0,2761.715771,736308.0,-26271.075117,-1.455192e-16,1995.742969,27503.045280,-27503.045280,736308.0,3.826052,5.0,-15.0,12.0,26.0,-2.0


In [21]:
# X_train1 = pd.DataFrame(X_train1).reset_index()
X_train1.columns = ['mean','std', 'min', 'max', 'kurt', 'skew','ptp','var','Rmean','Rstd','Rmax','Rmin','Imean','Istd','Imax','Imin','sum','mad','median','q.01','q.95','q.99','q.05']
X_train1.head()

,mean,std,min,max,kurt,skew,ptp,var,Rmean,Rstd,Rmax,Rmin,Imean,Istd,Imax,Imin,sum,mad,median,q.01,q.95,q.99,q.05
0,4.884113,5.101106,-98.0,104.0,33.662481,-0.024061,202.0,26.021284,12.0,2349.811482,732617.0,-20121.154171,-1.697723e-15,1399.854635,23432.719433,-23432.719433,732617.0,3.263401,5.0,-8.0,11.0,18.0,-2.0
1,4.725767,6.588824,-154.0,181.0,98.758517,0.390561,335.0,43.412599,5.0,2566.032248,708865.0,-31056.675076,1.843243e-15,1810.312266,27236.180586,-27236.180586,708865.0,3.574302,5.0,-11.0,12.0,21.0,-2.0
2,4.906393,6.967397,-106.0,140.0,33.555211,0.217391,246.0,48.544621,5.0,2683.549049,735959.0,-27654.557067,4.850638e-16,1921.220576,30073.497066,-30073.497066,735959.0,3.948411,5.0,-15.0,13.0,26.0,-3.0
3,4.902240,6.922305,-199.0,197.0,116.548172,0.757278,396.0,47.918309,5.0,2685.788525,735336.0,-25622.393604,-1.067140e-15,1891.826366,27380.321471,-27380.321471,735336.0,3.647117,5.0,-12.0,12.0,22.0,-2.0
4,4.908720,7.301110,-126.0,145.0,52.977905,0.064531,271.0,53.306210,12.0,2761.715771,736308.0,-26271.075117,-1.455192e-16,1995.742969,27503.045280,-27503.045280,736308.0,3.826052,5.0,-15.0,12.0,26.0,-2.0


In [22]:
# X_train1['rol_2']=X_train1['mean'].rolling(2).mean()
# X_train1['rol_3']=X_train1['mean'].rolling(3).mean()
# X_train1['rol_4']=X_train1['mean'].rolling(4).mean()
# X_train1['rol_5']=X_train1['mean'].rolling(5).mean()
# X_train1['rol_2']=X_train1['rol_2'].fillna(X_train1['rol_2'].dropna().mean())
# X_train1['rol_3']=X_train1['rol_2'].fillna(X_train1['rol_3'].dropna().mean())
# X_train1['rol_4']=X_train1['rol_4'].fillna(X_train1['rol_4'].dropna().mean())
# X_train1['rol_5']=X_train1['rol_5'].fillna(X_train1['rol_5'].dropna().mean())
# X_train1['rol_std_2']=X_train1['mean'].rolling(2).std()
# X_train1['rol_std_3']=X_train1['mean'].rolling(3).std()
# X_train1['rol_std_4']=X_train1['mean'].rolling(4).std()
# X_train1['rol_std_5']=X_train1['mean'].rolling(5).std()
# X_train1['rol_std_2']=X_train1['rol_std_2'].fillna(X_train1['rol_std_2'].dropna().mean())
# X_train1['rol_std_3']=X_train1['rol_std_3'].fillna(X_train1['rol_std_3'].dropna().mean())
# X_train1['rol_std_4']=X_train1['rol_std_4'].fillna(X_train1['rol_std_4'].dropna().mean())
# X_train1['rol_std_5']=X_train1['rol_std_5'].fillna(X_train1['rol_std_5'].dropna().mean())
X_train1['rol_2']=X_train1['mean'].rolling(2).mean()
X_train1['rol_2']=X_train1['rol_2'].fillna(X_train1['rol_2'].dropna().mean())

X_train1['rol_20']=X_train1['mean'].rolling(20).mean()
X_train1['rol_20']=X_train1['rol_20'].fillna(X_train1['rol_20'].dropna().mean())

X_train1'rol_10']=X_train1['mean'].rolling(10).mean()
X_train1['rol_10']=X_train1['rol_10'].fillna(X_train1['rol_10'].dropna().mean())
X_train1['rol_50']=X_train1['mean'].rolling(50).mean()
X_train1['rol_50']=X_train1['rol_50'].fillna(X_train1['rol_50'].dropna().mean())

SyntaxError: invalid syntax (<ipython-input-22-7356e818e82b>, line 23)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1, train_size=0.5, random_state=1234)


In [ ]:
model=XGBRegressor(iterations=100000, tree_depth=5, learning_rate=0.1, loss_function='MAE')
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [(value) for value in y_pred]
# evaluate predictions
MAE = mean_absolute_error(y_test, predictions)
MAE

In [ ]:

#to beat - 2.14738
y_pred=(model.predict(X_test))
mean_absolute_error(y_test, y_pred, sample_weight=None)

In [ ]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame()

# prepare test data
for seg_id in submission.index:
    seg = pd.read_csv('test/' + seg_id + '.csv')
    ch = gen_features(seg['acoustic_data'])
    X_test = X_test.append(ch, ignore_index=True)

X_test.columns = ['mean','std', 'min', 'max', 'kurt', 'skew','ptp','var','Rmean','Rstd','Rmax','Rmin','Imean','Istd','Imax','Imin','sum','mad','median','q.01','q.95','q.99','q.05']
# X_test = X_test.apply(LabelEncoder().fit_transform)
# X_test=X_test.drop(['skew,'], axis=1)
# X_test = pd.DataFrame(X_test).reset_index()



In [ ]:
#     X_test['rol_2']=X_test['mean'].rolling(2).mean()

#     X_test['rol_3']=X_test['mean'].rolling(3).mean()
#     X_test['rol_4']=X_test['mean'].rolling(4).mean()
#     X_test['rol_5']=X_test['mean'].rolling(5).mean()
#     X_test['rol_2']=X_test['rol_2'].fillna(X_test['rol_2'].dropna().mean())
#     X_test['rol_3']=X_test['rol_2'].fillna(X_test['rol_3'].dropna().mean())
#     X_test['rol_4']=X_test['rol_4'].fillna(X_test['rol_4'].dropna().mean())
#     X_test['rol_5']=X_test['rol_5'].fillna(X_test['rol_5'].dropna().mean())
#     X_test['rol_std_2']=X_test['mean'].rolling(2).std()
#     X_test['rol_std_3']=X_test['mean'].rolling(3).std()
#     X_test['rol_std_4']=X_test['mean'].rolling(4).std()
#     X_test['rol_std_5']=X_test['mean'].rolling(5).std()
#     X_test['rol_std_2']=X_test['rol_std_2'].fillna(X_test['rol_std_2'].dropna().mean())
#     X_test['rol_std_3']=X_test['rol_std_3'].fillna(X_test['rol_std_3'].dropna().mean())
#     X_test['rol_std_4']=X_test['rol_std_4'].fillna(X_test['rol_std_4'].dropna().mean())
#     X_test['rol_std_5']=X_test['rol_std_5'].fillna(X_test['rol_std_5'].dropna().mean())
X_test['rol_2']=X_test['mean'].rolling(2).mean()
X_test['rol_2']=X_train1['rol_2'].fillna(X_test['rol_2'].dropna().mean())
X_test['rol_20']=X_test['mean'].rolling(20).mean()
X_test['rol_20']=X_train1['rol_20'].fillna(X_test['rol_20'].dropna().mean())


X_test['rol_10']=X_test['mean'].rolling(10).mean()
X_test['rol_10']=X_test['rol_10'].fillna(X_test['rol_10'].dropna().mean())
X_test['rol_50']=X_test['mean'].rolling(50).mean()
X_test['rol_50']=X_test['rol_50'].fillna(X_test['rol_50'].dropna().mean())



In [ ]:
# X_test1 = Pool(X_test)
y_hat = model.predict(X_test)
submission['time_to_failure'] = y_hat
submission.to_csv('submission.csv')

In [ ]:
fold_importance_df=pd.DataFrame()
fold_importance_df['Feature']=X_train.columns
feature_importance_df=pd.DataFrame()
fold_importance_df["importance"]=model.feature_importances_[:len(X_train.columns)]
feature_importance_df=pd.concat([feature_importance_df, fold_importance_df], axis=0)
fold_importance_df.head()

In [ ]:

cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:200].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

# plt.figure(figsize=(14,26))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('CatBoost w_mean & rol20')
plt.tight_layout()
plt.savefig('catboost_w_mean_importances2.png')